# Gauss Thinning


In [ ]:
import igl
import meshplot as mp
import numpy as np
from numpy import linalg as LA
import math
from gauss_thinning import *


## Load and Plot Data


In [ ]:
# v, f = igl.read_triangle_mesh("examples/einstein/input.off")
# v, f = igl.read_triangle_mesh("examples/bumpy/input.off")
# v, f = igl.read_triangle_mesh("examples/bunny/input.off")
v, f = igl.read_triangle_mesh("examples/face/input.off")


## Visualizing Surfaces and Pointclouds


In [ ]:
mp.plot(v, f, c=v[:, 0])
mp.plot(v, c=v[:, 0], shading={"point_size": 0.3})


## Gauss map of the input mesh


In [ ]:
# Calculating per face normals
nf = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
# Plotting face normals
mp.plot(v, f, c=nf)


In [ ]:
# Calculating per vertex normals
nv = igl.per_vertex_normals(v, f)
avg = igl.avg_edge_length(v, f)
# Plotting vertex normals as lines
pv = mp.plot(v, f, c=nv)
pv.add_lines(v, v + nv * avg)


In [ ]:
# Calculating the principal curvature, mean curvature and gassian curvature.
v1, v2, k1, k2 = igl.principal_curvature(v, f)
mean_curv = 0.5 * (k1 + k2)
gaussian_curv = k1 * k2
avg = igl.avg_edge_length(v, f)

# Plotting gaussian curvature + directions
pgc = mp.plot(v, f, c=gaussian_curv)
pgc.add_lines(v + v1 * avg / 3, v - v1 * avg / 3, shading={"line_color": "red"})
pgc.add_lines(v + v2 * avg / 3, v - v2 * avg / 3, shading={"line_color": "green"})


In [ ]:
# Plotting the gauss map
vs, fs = igl.read_triangle_mesh("examples/unit_sphere.obj")
vs = vs * 0.95
pgm = mp.plot(vs, fs, shading={"flat": False})
pgm.add_points(nf, shading={"point_size": 0.2})
ev, fe, ef = igl.edge_topology(v, f)
ef = ef[np.all(ef != -1, axis=1)]
pgm.add_lines(nf[ef[:, 0]], nf[ef[:, 1]], shading={"line_color": "red"})


## Calculate the developability on the surface


In [ ]:
vf, ni = igl.vertex_triangle_adjacency(f, v.shape[0])


In [ ]:
developability = np.zeros(v.shape[0])
for i in range(0, v.shape[0]):
    dmat = nf[vf[ni[i] : ni[i + 1]]]
    developability[i] = min(np.linalg.eigvals(np.dot(dmat.T, dmat)))
pd = mp.plot(v, f, c=developability, shading={"point_size": 0.3})


In [ ]:
max(developability)


## Gauss Thinning

In [ ]:
v, f = igl.read_triangle_mesh("examples/bunny_small/input.off")
mp.plot(v, f, c=v[:, 0])
gauss_thinning(v, f, num_iterations=100, min_cone_angle=5, render=True)
